In [1]:
import ads


In [2]:
ads.__version__


'2.9.0'

## Aula  Lectura de la base de datos

In [1]:
from ads.dataset.factory import DatasetFactory

In [3]:
ds = DatasetFactory.open("data/datos_salud_final.csv")

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
ds.head()

In [ ]:
type(ds)

In [ ]:
ds.columns

In [ ]:
ds = DatasetFactory.open("datos_salud_final.csv", target="riesgo_eac_decada")
ds.head()

In [ ]:
type(ds)

In [ ]:
ds.target.show_in_notebook()

## 1.4 Creando nuestro primer modelo

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
train, test = ds.train_test_split()

In [ ]:
train.X

In [ ]:
test.X

In [ ]:
train.y

In [ ]:
test.y

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
primer_modelo = DecisionTreeClassifier()


In [ ]:
primer_modelo.fit(train.X, train.y)

In [ ]:
primer_modelo.score(test.X, test.y)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
matriz = ConfusionMatrixDisplay.from_predictions(estimetor=primer_modelo, X = test.X, y = test.y)

### 1.5 Mejorando el primer modelo

In [ ]:
modelo_profundo = DecisionTreeClassifier(max_depth=100)

In [ ]:
modelo_profundo.fit(train.X, train.y)

In [ ]:
modelo_profundo.score(test.X, test.y)

In [ ]:
matriz = ConfusionMatrixDisplay.from_estimator(estimetor=modelo_profundo, X = test.X, y = test.y)

### 1.6 Selección de Features

In [ ]:
atributos = primer_modelo.feature_importances_
atributos

In [ ]:
import pandas as pd

pd.Series(atributos, index=train.X.columns).sort_values()

In [ ]:
train.X.drop(columns="diabetes", axis=1, inplace=True)

In [ ]:
train.X.columns

In [ ]:
primer_modelo.fit(train.X, train.y)

In [ ]:
primer_modelo.score(test.X, test.y)

In [ ]:
matriz = ConfusionMatrixDisplay.from_estimator(estimetor=primer_modelo, X = test.X, y = test.y)

### 2.1 Explorando AutoML de Oracle

In [2]:
from ads.common.data import ADSData

In [ ]:
ads_data = ADSData.build(ds.to_pandas(),"riesgo_eac_decada")
type(ads_data)

In [ ]:
from ads.automl.driver import AutoML

In [ ]:
oracle_automl = AutoML(ads_data)

In [ ]:
modelo_automl, baseline = oracle_automl.train(score_metric="accuracy", model_list=["DecisionTreeClassifier", 
                                                                                   "SVC", 
                                                                                   "logisticRegression",
                                                                                   "RandomForestClassifier"],
                                                                                   time_budget=150)

In [ ]:
oracle_automl.print_trials(max_row=20, sort_column="Mean Validation Score")

In [ ]:
oracle_automl.visualize_algorithm_selection_trials()

### 2.3 Features y modelo seleccionado

In [ ]:
modelo_automl.show_in_notebook()

In [ ]:
oracle_automl.visualize_algorithm_selection_trials()

### 3.1 ADS Evaluator

In [ ]:
modelo_automl.feature_names()

In [ ]:
train, test = ds[['sexo', 'edad', 'fumador', 'cigarrillos_por_dia',
       'uso_medicamento_presion', 'acv', 'hipertension', 'colesterol_total',
       'presion_arterial_sistolica', 'presion_arterial_diastolica', 'imc',
       'frecuencia_cardiaca', 'glicemia', 'categoria_de_fumador', "riesgo_eaac_decada"]].train_test_split()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
modelo_automl.summary()

In [ ]:
parametros = {"se usa el dicionario"}

In [ ]:
mejor_modelo = RandomForestClassifier(**parametros)

In [ ]:
primer_modelo = DecisionTreeClassifier(max_depth=100)

In [ ]:
mejor_modelo.fit(train.X, train.y)
primer_modelo.fit(train.X, train.y)

In [ ]:
from ads.common.model import ADSModel

In [ ]:
ads_mejor_modelo = ADSModel.from_estimator(mejor_modelo, name="Mejor Modelo")
ads_primer_modelo = ADSModel.from_estimator(primer_modelo, name="Primer Modelo")

In [ ]:
from ads.evaluations.evaluator import ADSEvaluator

In [ ]:
evaluator = ADSEvaluator(test_data=test, models=[ads_mejor_modelo, ads_primer_modelo], training_data=train)

### 3.2 Analizando las metricas

In [ ]:
evaluator.show_in_notebook()

#### 3.3 Evaluando más métricas

In [ ]:
evaluator.metrics

### 3.4 Overfitting

In [ ]:
primer_modelo = DecisionTreeClassifier(max_depth=5)
primer_modelo.fit(train.X, train.y)
ads_primer_modelo = ADSModel.from_estimator(primer_modelo, name="Primer Modelo")

In [ ]:
evaluator = ADSEvaluator(test_data=test, models=[ads_mejor_modelo, ads_primer_modelo], training_data=train)

In [ ]:
evaluator.metrics

### 4.1 Model Explainability

In [ ]:
from ads.explanations.explainer import ADSExplainer

In [ ]:
explainer = ADSExplainer(test, ads_mejor_modelo, train)

In [ ]:
explainer_global = explainer.global_explanation()

In [ ]:
importance = explainer_global.compute_feature_importance()

In [ ]:
importance.show_in_notebook("detailed")

### 4.2 Valores de Features x Clasificación

In [ ]:
pdp_edad = explainer_global.compute_partial_dependence("edad")

In [ ]:
pdp_presion = explainer_global.compute_partial_dependence("presion_arterial_diastolica")

In [ ]:
pdp_presion.show_in_notebook()

In [ ]:
pdp_edad.show_in_notebook()

### 4.3 Explicando decisión individual

In [ ]:
local_explainer = explainer.local_explanation()

In [ ]:
test.X.iloc[[13]]

In [ ]:
test.y.iloc[[13]]

In [ ]:
explanation = local_explainer.explain(test.X.iloc[[13]], test.y.iloc[[13]])

In [ ]:
explanation.show_in_notebook()

### 5.1 Creando nuestro artefacto